In [ ]:
import pandas as pd
from IPython.display import display
from tensorflow import keras
from tensorflow.keras import layers, callbacks

In [ ]:
#import datasets
red_wine = pd.read_csv('./data/winequality-white.csv', sep=';')

In [ ]:
# Create training and validation splits
df_train = red_wine.sample(frac=0.8, random_state=0)
df_valid = red_wine.drop(df_train.index)
display(df_train.head(4))

In [ ]:
# Scale to [0, 1]
max_ = df_train.max(axis=0)
min_ = df_train.min(axis=0)
df_train = (df_train - min_)/(max_ - min_)
df_valid = (df_valid - min_)/(max_ - min_)

In [ ]:
# Split features and target
X_train = df_train.drop('quality', axis=1)
X_valid = df_valid.drop('quality', axis=1)
y_train = df_train['quality']
y_valid = df_valid['quality']
print(X_train.shape)

In [ ]:
# Early stopping criteria
early_stopping = callbacks.EarlyStopping(
    min_delta=0.01, # minimium amount of change to count as an improvement
    patience=100, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
# Build a Sequential Model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[11]), # Input layer
    layers.Dense(64, activation='relu'), # 1st hidden layer
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'), # 2nd hidden layer
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'), # 3rd hidden layer
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'), # 4th hidden layer
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(64, activation='relu'), # 5th hidden layer
    layers.Dense(1), # output layer
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mae',)

In [ ]:
# Run the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=4096,
    epochs=500,
    verbose=1
)

In [ ]:
# convert the training history to a dataframe
history_df = pd.DataFrame(history.history)
# use Pandas native plot method
# Show the learning curves
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot();

In [ ]:
print("Minimum validation loss: {}".format(history_df['val_loss'].min()))